In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('../02_dataCreated/newData20170830.csv')

/home/ram22/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (14,37,38,75,76,77,78,79,80,81,85,87,89,90,91,92,93,97,101,105,107,108,109,127,128,135,136,137,138,141,143,145,146,147,150,151,152,153,154,155,156,159,161,172,173,175,176,177,179,180,181,183,184,185,187,193,195,197,199,201,203,206,208,210,215) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.loc[df['shipper_declared'] == 'NOT DECLARED', 'shipper_unified_id'] = np.nan 
# all cases where shipper id is unified are also shipper delclared == NOT DECLARED
df.loc[df['calc_value_usd'] == 0, 'calc_value_usd'] = np.nan 
df.loc[df['calc_weight_kg'] == 0, 'calc_weight_kg'] = np.nan 

In [4]:
temp = df['hs_code'].apply(lambda x: list(re.split(r'\s*,\s*',
                 x.replace('-', ',').replace(
                     'nov', '11').replace('ago', '08').replace('ND', '00').replace('EC', '00')))) 
df.loc[df['hs_code'].str.find('00') > -1, 'hs_code'] = np.nan

In [5]:
fieldsToCheck = ['country_of_origin', 'shipper_unified_id', 'calc_value_usd', 'calc_weight_kg', 'hs_code']

In [6]:
dfMiss = pd.concat([df['consignee_d_b_business_name'], df[fieldsToCheck].notnull()], axis=1)

In [7]:
dfMiss.head()

,consignee_d_b_business_name,country_of_origin,shipper_unified_id,calc_value_usd,calc_weight_kg,hs_code
0,E. I. DU PONT DE NEMOURS AND COMPANY,True,True,True,True,True
1,E. I. DU PONT DE NEMOURS AND COMPANY,True,True,True,True,True
2,E. I. DU PONT DE NEMOURS AND COMPANY,True,True,True,True,True
3,MITSUBISHI CORPORATION,True,True,True,True,True
4,"HENRY SCHEIN, INC.",True,True,False,True,False


In [8]:
missGrouped = dfMiss.groupby('consignee_d_b_business_name')

In [9]:
part1 = pd.DataFrame(missGrouped['country_of_origin'].count()).rename(columns={'country_of_origin': 'total_records'})
part2 = missGrouped.sum()
mergedCounts = part1.merge(part2, left_index=True, right_index=True)

In [10]:
complete = mergedCounts[['country_of_origin', 'shipper_unified_id',
       'calc_value_usd', 'calc_weight_kg', 'hs_code']].divide(mergedCounts['total_records'], axis=0)
complete = complete.merge(pd.DataFrame(mergedCounts['total_records']), left_index=True, right_index=True)

In [11]:
complete

,country_of_origin,shipper_unified_id,calc_value_usd,calc_weight_kg,hs_code,total_records
consignee_d_b_business_name,,,,,,
BAE SYSTEMS PLC,1.0,0.995575,0.716814,0.995575,0.814159,226
E. I. DU PONT DE NEMOURS AND COMPANY,1.0,0.647551,0.792946,0.996995,0.899020,7655
GLAXOSMITHKLINE PLC,1.0,1.000000,0.921875,1.000000,0.940104,384
"HENRY SCHEIN, INC.",1.0,0.873405,0.725221,1.000000,0.881256,1019
"IBERDROLA, SOCIEDAD ANONIMA",1.0,0.956835,0.884892,1.000000,0.971223,139
INDORAMA VENTURES PUBLIC COMPANY LIMITED,1.0,1.000000,0.889299,1.000000,0.963100,271
LOUIS DREYFUS HOLDING B.V.,1.0,0.973420,0.942626,1.000000,0.990600,3085
MICROSOFT CORPORATION,1.0,0.854558,0.910974,0.999762,0.957629,4201
MITSUBISHI CORPORATION,1.0,0.956724,0.867127,0.999112,0.934857,39421


In [12]:
allComplete = df[['consignee_d_b_business_name', 'country_of_origin']][df[fieldsToCheck].notnull().all(axis=1)].groupby(
    'consignee_d_b_business_name').count().rename(columns={'country_of_origin' : 'all_complete'})
allComplete

,all_complete
consignee_d_b_business_name,
BAE SYSTEMS PLC,161
E. I. DU PONT DE NEMOURS AND COMPANY,3971
GLAXOSMITHKLINE PLC,354
"HENRY SCHEIN, INC.",628
"IBERDROLA, SOCIEDAD ANONIMA",117
INDORAMA VENTURES PUBLIC COMPANY LIMITED,241
LOUIS DREYFUS HOLDING B.V.,2826
MICROSOFT CORPORATION,3240
MITSUBISHI CORPORATION,32640


In [13]:
complete['all_complete'] = allComplete['all_complete'] / complete['total_records']

In [14]:
complete.loc[complete['all_complete'].isnull(), 'all_complete'] = 0

In [15]:
complete = complete[['total_records', 'all_complete', 'country_of_origin', 'shipper_unified_id', 'calc_value_usd', 'calc_weight_kg', 'hs_code']]

In [16]:
complete.describe()

,total_records,all_complete,country_of_origin,shipper_unified_id,calc_value_usd,calc_weight_kg,hs_code
count,19.000000,19.000000,19.0,19.000000,19.000000,19.000000,19.000000
mean,4943.631579,0.774166,1.0,0.928122,0.837807,0.992870,0.907084
std,9624.831550,0.136725,0.0,0.105382,0.103276,0.028567,0.082356
min,8.000000,0.518746,1.0,0.647551,0.619048,0.875000,0.690476
25%,99.500000,0.672861,1.0,0.912440,0.771473,0.999325,0.878128
50%,1019.000000,0.827869,1.0,0.962963,0.876939,1.000000,0.934857
75%,4846.500000,0.870575,1.0,0.997788,0.904491,1.000000,0.964534
max,39421.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000


In [17]:
complete.to_csv('completeness.csv')

In [18]:
complete

,total_records,all_complete,country_of_origin,shipper_unified_id,calc_value_usd,calc_weight_kg,hs_code
consignee_d_b_business_name,,,,,,,
BAE SYSTEMS PLC,226,0.712389,1.0,0.995575,0.716814,0.995575,0.814159
E. I. DU PONT DE NEMOURS AND COMPANY,7655,0.518746,1.0,0.647551,0.792946,0.996995,0.899020
GLAXOSMITHKLINE PLC,384,0.921875,1.0,1.000000,0.921875,1.000000,0.940104
"HENRY SCHEIN, INC.",1019,0.616290,1.0,0.873405,0.725221,1.000000,0.881256
"IBERDROLA, SOCIEDAD ANONIMA",139,0.841727,1.0,0.956835,0.884892,1.000000,0.971223
INDORAMA VENTURES PUBLIC COMPANY LIMITED,271,0.889299,1.0,1.000000,0.889299,1.000000,0.963100
LOUIS DREYFUS HOLDING B.V.,3085,0.916045,1.0,0.973420,0.942626,1.000000,0.990600
MICROSOFT CORPORATION,4201,0.771245,1.0,0.854558,0.910974,0.999762,0.957629
MITSUBISHI CORPORATION,39421,0.827985,1.0,0.956724,0.867127,0.999112,0.934857
